An example of simulating $\eta$ meson production in $pp$ collisisons, followed by their decay into photons and dark vectors

In [1]:
# this is based on main01.py example in the PYTHIA distribution
# PYTHIA must be compiled with the python interface enabled to make this notebook work

import numpy as np
import sys
import mc
import kinematics as kin

# Import the Pythia module 
lib = '/home/nblinov/HEP/pythia8306/lib'
sys.path.insert(0, lib)

import pythia8

def generate_meson_decay_chain(p_meson, m_vector, mf):
    """
    Generate decay chain meson > gamma + V, V > f f
    Args:
        p_meson - four-momentum of the parent meson
        m_vector - dark vector mass in the decay meson > gamma + V
        mf - final state mass in the dark vector decay V > f f
    Returns:
        list of four vectors [p_meson, p_gamma, p_vector, p1, p2]
    """
    m_meson = np.sqrt(kin.lor_prod(p_meson,p_meson))
    p_gamma, p_vector  = mc.gen_two_body_decay_products(p_meson, m_meson, 0., m_vector)
    p1, p2 = mc.gen_two_body_decay_products(p_vector, m_vector, mf, mf)
    
    return [p_meson, p_gamma, p_vector, p1, p2]


Generate some mesons from LHC collisions, while disabling $\eta$ decays, so that we can collect them and decay by hand

In [3]:
pythia = pythia8.Pythia()
pythia.readString("Beams:eCM = 14000.")

#pi0
#pythia.readString("111: mayDecay = false");
#eta
pythia.readString("221: mayDecay = false");

#pythia.readString("SoftQCD:all = on");
pythia.readString("HardQCD:all = on");

pythia.readString("PhaseSpace:pTHatMin = 20.")
nEvents = 10


pythia.init()


pi0_total = 0
eta_total = 0
eta_prime_total = 0

pi0_four_vector_list = []
eta_four_vector_list = []
eta_prime_four_vector_list = []


# Begin event loop. Generate event. Skip if error. List first one.
for iEvent in range(0, nEvents):
    if not pythia.next(): continue

    for prt in pythia.event:
        if prt.isFinal() and prt.id() == 111:
            pi0_total += 1
            pi0_four_vector_list.append(np.array([prt.e(), prt.px(), prt.py(), prt.pz()]))

        if prt.isFinal() and prt.id() == 221:
            eta_total += 1
            eta_four_vector_list.append(np.array([prt.e(), prt.px(), prt.py(), prt.pz()]))
         
        if prt.isFinal() and prt.id() == 331:
            eta_prime_total += 1
            eta_prime_four_vector_list.append(np.array([prt.e(), prt.px(), prt.py(), prt.pz()]))

# End of event loop. Statistics. Histogram. Done.
pythia.stat();

print("Total # of pi0s = ", pi0_total)
print("Total # of etas = ", eta_total)
print("Total # of etas primes = ", eta_prime_total)
print("pi0 per interaction = ", pi0_total/nEvents)
print("eta per interaction = ", eta_total/nEvents)
print("eta prime per interactio = ", eta_prime_total/nEvents)

pi0_four_vector_list = np.array(pi0_four_vector_list)
eta_four_vector_list = np.array(eta_four_vector_list)
eta_prime_four_vector_list = np.array(eta_prime_four_vector_list)

Total # of pi0s =  0
Total # of etas =  125
Total # of etas primes =  0
pi0 per interaction =  0.0
eta per interaction =  12.5
eta prime per interactio =  0.0


Decay the collected $\eta$s as $\eta \to \gamma V$, with $V\to \mu^+ \mu^-$ 

In [4]:
m_vector = 0.3
mf = 0.105
eta_decay_events = [generate_meson_decay_chain(p_meson, m_vector, mf) for p_meson in eta_four_vector_list]

In [5]:
# particle order: [p_meson, p_gamma, p_vector, p_fermion1, p_fermion2]
eta_decay_events[0]

[array([ 23.27519004,   1.23457619,  -0.08200269, -23.23582211]),
 array([ 8.64583744,  0.26834723, -0.04897424, -8.64153321]),
 array([ 14.62935261,   0.96622896,  -0.03302845, -14.5942889 ]),
 array([ 6.54754078,  0.40087183, -0.11591658, -6.53338583]),
 array([ 8.08181182,  0.56535713,  0.08288813, -8.06090306])]

Check momentum conservation

In [6]:
print("m_eta = ", np.sqrt(kin.lor_prod(eta_decay_events[0][0],eta_decay_events[0][0])))
print("p_eta - p_vector - p_gamma = ", eta_decay_events[0][0] - eta_decay_events[0][1]-eta_decay_events[0][2])
print("m_vector = ", np.sqrt(kin.lor_prod(eta_decay_events[0][2],eta_decay_events[0][2])))
print("p_vector - p1 - p2 = ", eta_decay_events[0][2] - eta_decay_events[0][3]-eta_decay_events[0][4])
print("m_fermion = ", np.sqrt(kin.lor_prod(eta_decay_events[0][4],eta_decay_events[0][4])))

m_eta =  0.5478500000001667
p_eta - p_vector - p_gamma =  [ 1.00008890e-12  5.31796829e-14 -3.52495810e-15 -9.96536187e-13]
m_vector =  0.3000000000000057
p_vector - p1 - p2 =  [-5.06616971e-12 -3.34621220e-13  1.14352972e-14  5.05551156e-12]
m_fermion =  0.10500000000001722
